<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-0222-dev/CORS_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CORS Functions.
(c) 2020, C. W. Wright

Gather RINEX data from ftp://geodesy.noaa.gov/cors/rinex/

Data older than 30 days at sampling rate can be had by contacting: https://www.avl.class.noaa.gov/saa/products/search?datatype_family=CORS

In [ ]:
#@title imports: Fetch CORS GNSS data for 1 or more sites on a date.


import datetime
import time
import os
import re
import glob
import pandas as pd
import multiprocessing
import ipywidgets as ipw
import subprocess as sp


try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False


In [ ]:
#@title def teqc_install()  {form-width: "35%"} 
# Get and install teqc
def teqc_install():
  rv = sp.run(f'which teqc', shell=True)
  os.chdir('/content/')
  if rv == []:
    print('Download and install Teqc from Unavco.')
    sp.run(f'wget https://www.unavco.org/software/data-processing/teqc/development/teqc_Lx86_64s.zip', shell=True)
    sp.run(f'unzip teqc_Lx86_64s.zip', shell=True)
    sp.run(f'mv teqc /usr/local/bin', shell=True)
    sp.run(f'rm -rf teqc_Lx86_64s.zip', shell=True)
  else:
    a = 1;
    #print('    Teqc was already installed.')



In [ ]:
#@title def extract_year( date ) {form-width: "35%"}
def extract_year( date ):
  Year = date.split('/')[0]
  return Year




In [ ]:
#@title def isTimeFormat(ts)  {form-width: "35%"}

def isTimeFormat(ts):
  """
  isTimeFormat(ts) checks the 'ts' input string for validity. A valid string is in
  this format: '13:34:56'.

  Returns True for a valid string,a nd False for invalid.
  
  """
  t = []
  rv = False;
  try:
      t = time.strptime(ts, '%H:%M:%S')
      rv = True;
  except ValueError:
      rv = False
  return rv

# testing code...
#if __name__ == '__main__':
#  for t in ['12:34:56',
#            '12:34',
#            '01:02:03',
#            '23:01:02',
#            '25:34:63']:
#    print(f'rv:{isTimeFormat(t)}')

In [ ]:
#@title def compute_day_of_year(date_str)
def compute_day_of_year(date_str):
  Day = int(( datetime.datetime.strptime(date_str,'%Y/%m/%d') - datetime.datetime(int(date_str.split('/')[0]),1,1)).days + 1 )
  Day = f'{Day:03d}'
  return Day


In [ ]:
#@title def CORS_get_all_station_data()  {form-width: "35%"}

def CORS_get_all_station_data(Project_Root_Folder, CORS_to_Fetch, date, UTC_Start_Time, UTC_End_Time):
  teqc_install()
  print(f'Fetching CORS Data for: {date}')
  Date = date.replace('-', '/')
  CORS_to_Fetch = CORS_to_Fetch.lower()
  CORS_to_Fetch = CORS_to_Fetch.replace(',', ' ')
  lst = CORS_to_Fetch.replace(',', ' ').split()
  get_CORS_SP3_Nav(Project_Root_Folder, Date )
  for sta in lst:
    rv = get_CORS_OBS_Data(Project_Root_Folder, Date, sta, UTC_Start_Time, UTC_End_Time )
    CORS_get_station_coords(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
    CORS_get_station_datasheet(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
  clean_up_CORS(Project_Root_Folder, Date)
  print('All requested CORS data downloaded.  Operation completed.')



In [ ]:
#@title def get_CORS_OBS_Data( rootdir, date, Station, start, stop ) {form-width: "35%"}
def get_CORS_OBS_Data( rootdir, date, Station, start, stop ):
  """
  inputs: string rootdir,     Path where to store the CORS data.
          string date,        Date to fetch. ex: '2019/1/1'
          string Station,     CORS Station to fetch.
          string start,       Start time, ex '12:00:00'
          string stop         End time, ex. '15:30:01'
  returns
    Nothing.  Fetches data via ftp and places in local directory structure
  
  Example: 
    get_CORS_OBS_Data( '/content/2019-data', '2019/11/26', 'ncdu', '12:00:00', '15:30:01' )

  """
  Base_Date = date
  Day = compute_day_of_year(Base_Date)
  #Day = int(( datetime.datetime.strptime(Base_Date,'%Y/%m/%d') - datetime.datetime(int(Base_Date.split('/')[0]),1,1)).days + 1 )
  #Day = f'{Day:03d}'
  Base_FTP_Site = "ftp://geodesy.noaa.gov/cors/rinex/"
  try:
    os.makedirs(rootdir, exist_ok=True)
  except:
    print(f'{rootdir} already exists.')

  rootdir = rootdir+'/'
  Base_root = 'CORS'
  Year = extract_year(date)
  Year2 = Year[2:4]
  YD = Year+'/'+Day
  print(f'Gathering Data for: {Station}  for: {Base_Date}')
  # print(YD)
  print(sp.run(f'mkdir -p {rootdir}', shell=True))
  os.chdir(rootdir)
  print(sp.run(f'mkdir -p {Base_root}', shell=True))
  print(sp.run(f'mkdir -p {Base_root}/{Station}', shell=True))
  d = rootdir+Base_root+'/'+Station
  print(f'd={d}')
  os.chdir(d)
  url = f'{Base_FTP_Site}/{YD}/{Station}/{Station+str(Day)+"0"}.*'
  print(f'url: {url}')
  #os.chdir(str(rootdir+Base_root))
  #os.chdir(rootdir)
  os.chdir(d)
  rv = glob.glob('*.*o')
  print(f'Pre-existing RINEX observation files found: {rv}')
  if (rv) != []:
    print(f'OBS Files exist.  Skipped. rv:{rv}')
  else:
    print(sp.run(f'wget -nv -nd -r --no-clobber {url}', shell=True))
    print(sp.run(f'gunzip -qr .', shell=True))
    obsfn = rootdir+Base_root+'/'+Station+'/'+Station+str(Day)+f'0.{Year2}o'
    print(f'Requested obs file: {obsfn}' )
    CORS_trim_to_time( obsfn, start, stop)
    print( f'Download of {Station} completed.')



In [ ]:
#!cd /content/xx; wget -nv -nd -r --no-clobber ftp://geodesy.noaa.gov/cors/rinex//2020/173/ncdu/ncdu173[a-z].20o.gz
!!cd /content/xx; 

In [ ]:
#@title def get_CORS_SP3_Nav( rootdir,  date ) {form-width: "10%"}
def get_CORS_SP3_Nav( rootdir,  date ):
  """
  get_CORS_SP3_Nav( rootdir,  date )

  inputs: string rootdir,     Path where to store the CORS data.
          string date         Date to fetch. ex: '2019/1/1'

  returns
    Nothing.  Fetches SP3 data via ftp and places in local directory structure

  get_CORS_SP3_Nav() gathers all of the sp3 files for a given date.  Sp3 files contain the
  GPS Satellite Ephemerides / Satellite & Station Clocks.  More information on these files
  can be found at: http://www.igs.org/products
  
  Example: 
    def get_CORS_SP3_Nav( '/content/2019-data', '2019/11/26' )


  """
  Base_Date = date
  Day = compute_day_of_year(Base_Date)
  #Day = int(( datetime.datetime.strptime(Base_Date,'%Y/%m/%d') - datetime.datetime(int(Base_Date.split('/')[0]),1,1)).days + 1 )
  #Day = f'{Day:03d}'
  Base_FTP_Site = "ftp://geodesy.noaa.gov/cors/rinex/"
  sp.run( f'mkdir -p {rootdir}', shell=True )
  rootdir = rootdir+'/'
  Base_root = 'CORS'
  Year = extract_year(date)
  YD = Year+'/'+Day
  print(YD)
  os.chdir(rootdir)
  sp.run( f'mkdir -p {Base_root}', shell=True)
  os.chdir(str(rootdir+Base_root))
  sp.run(f'mkdir -p sp3 nav', shell=True)
  os.chdir('sp3')
  print(f'   Gathering Sp3 Data for: {Base_Date}')
  sp.run( f'wget -nv -nd -r {Base_FTP_Site}/{YD}/*.sp3.gz', shell=True)
  os.chdir('../nav') 
  print(f'   Gathering Nav Data for: {Base_Date}')
  sp.run(f'wget -nv -nd -r {Base_FTP_Site}/{YD}/*[ng].gz', shell=True)
  os.chdir(rootdir)
  sp.run( f'gunzip -qrf .', shell=True)
  print( f'Download of {date} Nav & SP3 completed.')



In [ ]:
#@title def CORS_get_station_datasheet(dir, station) {form-width: "10%"}

# url: https://geodesy.noaa.gov/cgi-bin/ds_cors.prl?CorsSelected=|NCDU&CorsTypeSelected=Arp
def CORS_get_station_datasheet(dir, station):
  station = str.lower(station)
  print(station)
  os.makedirs(dir, exist_ok=True)
  rv = sp.run(f'curl  -B https://geodesy.noaa.gov/cgi-bin/ds_cors.prl?CorsSelected=|{str.upper(station)}&CorsTypeSelected=Arp > {dir}/{station}_datasheet.html', shell=True)
  return rv



In [ ]:
#@title def CORS_get_station_coords(dir, station) {form-width: "10%"}
# url for data:
def CORS_get_station_coords(dir, station):
  station = str.lower(station)
  os.makedirs(dir, exist_ok=True)
  rv = sp.run(f'curl  -B ftp://geodesy.noaa.gov/cors/coord/coord_14/{station}_14.coord.txt > {dir}/{station}_14.coord.txt', shell=True)


In [ ]:
#@title def clean_up_CORS(rdir, date) {form-width: "10%"}
def clean_up_CORS(rdir, date):
  """
  clean_up_CORS(rdir, date)
  Inputs:
  rdir      Directory to start the find command in.
  date      The date of the CORS files to cleanup.

  Outputs:
            Removes unnecessary files.
  Returns:
            None.

  Uses the Linux 'find' command to locate and remove unnecessary files from within a CORS station directory.  
  """
  Year = date.split('/')[0][2:4]
  sp.check_output(f'cd {rdir}; find -name *.md5 -delete; find -name *.{Year}d -delete; find -name *.md5* -delete; find -name *.{Year}S* -delete;', shell=True) 



In [ ]:
#@title def CORS_trim_to_time(f, start, stop) {form-width: "35%"}
# Url for data: ftp://geodesy.noaa.gov/cors/coord/coord_14/
def CORS_trim_to_time(f, start, stop):
  """
  CORS_trim_to_time(f, start, stop)

  Inputs:
  f       RINEX File to trim.
  start   UTC Start time string. Ex '12:00:05'
  stop    UTC Stop  time string. Ex '12:00:05'

  Outputs:
  Overwrites the input file with the new trimmed version.

  Returns:
    None.

Trims a RINEX file (f) to be between the 'start' and 'stop' times given.
  """
  st = re.sub('[\:\-\.]', '', start)
  end = re.sub('[\:\-\.]', '', stop)
  print(f'f: {f}   start: {start}  stop: {stop}')
  sp.run(f'teqc +out tmp.txt -st {start} -e {stop} {f}', shell=True)
  sp.run(f'mv tmp.txt {f}', shell=True)



# Testing & Example code.

In [ ]:
#@title  (Jupyter Widgets) CORS_jw() {form-width: "25%"}
import numpy as np
import datetime



def CORS_jw():
  """
  CORS_jw() Uses Juypter Widget based GUI to download and save CORS RINEX data from NOAA. 

  CORS Site:  https://geodesy.noaa.gov/CORS/
  Juypter Widgets: https://ipywidgets.readthedocs.io/en/latest/index.html
  """
  #@title def GET_sp3(b) {form-width: "25%"}
  # Reponse to SP3 button push
  def GET_sp3(b):
    print(f'GET_sp3 b={b}')
    global CORS_Date
    stations = sta_list.value
    CORS_Date =  date_w.value
    CORS_Date_str = CORS_Date.strftime("%Y/%m/%d")
    rv = get_CORS_SP3_Nav( proj_dir.value,  CORS_Date_str )

  #@title def GET_CORS(b)  {form-width: "25%"}
  # Respone to Download RINEX button
  def GET_CORS(b):
    global CORS_Date
    stations = sta_list.value
    CORS_Date =  date_w.value
    CORS_Date_str = CORS_Date.strftime("%Y/%m/%d")
    Start_time_str = Start_time.value
    End_time_str = End_time.value
    with output:
      if isTimeFormat(Start_time_str) == False:
        print(f'{Start_time_str} is not a valid start time. The correct format is: "HH:MM:SS" ')
        return
      if isTimeFormat(End_time_str) == False:
        print(f'{End_time_str} is not a valid end time.  The correct format is: "HH:MM:SS"')
        return
      print(f'stations: {stations}')
      print(f'Project_dir:{proj_dir.value}')
      print(f'Date: {CORS_Date} {type(CORS_Date)}  {CORS_Date_str}')
      print(f'Start Time:{Start_time_str}')
      print(f'End Time:{End_time_str}')
      CORS_get_all_station_data(proj_dir.value, stations,  CORS_Date_str, Start_time_str, End_time_str)

  try:
    CORS_Date
  except:
    CORS_Date = datetime.datetime.now()

  try:
    Start_time_str
  except:
    Start_time_str = '00:00:00'

  try:
    End_time_str
  except:
    End_time_str = '23:59:59'
  # Generate Jupyter Widgets entry widgets 
  proj_dir_lbl = ipw.Label( value="Project Path:")
  proj_dir = ipw.Text(value='/content/', layout=ipw.Layout(width='590px'))
  proj_dir_w = ipw.HBox( [proj_dir_lbl, proj_dir] )

  # same for Station list
  sta_lbl = ipw.Label(value="CORS Stations:")
  sta_list = ipw.Text(value='ncdu ncbe', disabled=False, layout=ipw.Layout(width='580px'))
  pd = ipw.HBox( [sta_lbl, sta_list  ])

  # generate a date picker widget and preload with CORS_Date if it exists
  date_w = ipw.DatePicker( description='Date:', disabled=False, value=CORS_Date, layout=ipw.Layout(width='250px') )

  # Generate Start and Stop time GUI entries
  Start_lbl  = ipw.Label( value="UTC Time Start:", layout=ipw.Layout(width='100px'))
  Start_time = ipw.Text(value=Start_time_str, layout=ipw.Layout(width='100px'))
  End_lbl    = ipw.Label( value="UTC End Time:", layout=ipw.Layout(width='100px'))
  End_time   = ipw.Text(value=End_time_str, layout=ipw.Layout(width='100px'))
  b = ipw.HBox([date_w, Start_lbl, Start_time, End_lbl, End_time])

  # Buttons
  sp3_b = ipw.Button(description = "Download SP3 Data", layout=ipw.Layout(width='180px') )
  sp3_b.style.button_color = 'lightgreen'

  go = ipw.Button(description = "Download CORS RINEX Data", layout=ipw.Layout(width='200px') )
  go.style.button_color = 'lightblue'
  buttons_w = ipw.Box([sp3_b, go])
  output = ipw.Output()
  go.on_click(GET_CORS)
  sp3_b.on_click(GET_sp3)
  display( ipw.HTML(value="<h1>Download CORS Data.</h1>"), pd, b, proj_dir_w, buttons_w, output)


if __name__ == '__main__':
  CORS_jw();



In [ ]:
if __name__ == '__main__':
  ! head -50 /content/CORS/ncdu/ncdu1540.20o

In [ ]:
#@title  Test CORS_get_all_station_data(Project_Root_Folder, CORS_to_Fetch, date, UTC_Start_Time, UTC_End_Time) {form-width: "35%"}
if __name__ == '__main__':
  CORS_to_Fetch = "ls03,    ncdu ncbi"
  CORS_get_all_station_data('/content/2020-0524', CORS_to_Fetch,  '2020/05/24', '12:00:00', '13:31:01')


In [ ]:
#@title Test (Colabs) CORS_get_station_datasheet() {form-width: "35%"}
if __name__ == '__main__':
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  CORS_Station = "NCDU" #@param {type:"string"}
  CORS_Station = str.lower(CORS_Station)
  d = f'{Project_Root_Folder}/CORS/{CORS_Station}/'
  rv = CORS_get_station_datasheet(d, CORS_Station)

In [ ]:
#@title Test CORS_get_station_coords() {form-width: "35%"}
if __name__ == '__main__':
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  CORS_Station = "NCDU" #@param {type:"string"}
  CORS_Station = str.lower(CORS_Station)
  d = f'{Project_Root_Folder}/CORS/{CORS_Station}/'
  CORS_get_station_coords('/content/2020-0524/CORS/ncdu', 'NCDU')

In [ ]:
#@title Test Google Colabs test code below getting all requested data for a site. {form-width: "35%"}


if __name__ == '__main__':
  teqc_install()
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  #@markdown Enter a list of CORS Stations to download below. Separate with "," or space character.
  CORS_to_Fetch = "ls03,    ncdu ncbi ncbx ncci ncbe ncso" #@param {type:"string"}
  Date_to_Fetch = "2020-05-24" #@param {type:"date"}
  UTC_Start_Time = "16:32:20" #@param {type:"string"}
  UTC_End_Time   = "18:50:31" #@param {type:"string"}

  print(f'Fetching CORS Data for: {Date_to_Fetch}')
  Date = Date_to_Fetch.replace('-', '/')
  CORS_to_Fetch = CORS_to_Fetch.lower()
  CORS_to_Fetch = CORS_to_Fetch.replace(',', ' ')
  lst = CORS_to_Fetch.replace(',', ' ').split()
  get_CORS_SP3_Nav(Project_Root_Folder, Date )
  for sta in lst:
    rv = get_CORS_OBS_Data(Project_Root_Folder, Date, sta, UTC_Start_Time, UTC_End_Time )
    CORS_get_station_coords(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
    CORS_get_station_datasheet(Project_Root_Folder+'/'+'CORS/'+sta+'/', sta)
  clean_up_CORS(Project_Root_Folder, Date)
  print('All requested CORS data downloaded.  Operation completed.')

#--------------------------------------------------------



In [ ]:
#@title Test Google Colabs test getting sp3 files.
if __name__ == '__main__':
  teqc_install()
  Project_Root_Folder = "/content/2020-0524" #@param {type:"string"}
  Date_to_Fetch = "2020/05/24" #@param {type:"date"}
  rv = get_CORS_SP3_Nav(Project_Root_Folder, Date_to_Fetch )


In [ ]:
#@title Display the first 40 lines of a file.
if __name__ == '__main__':
  File = "/content/2020-0524/CORS/ncdu/ncdu1450.20o" #@param {type:"string"}
  !head -40 {File}

# References, Todo, Change Log & Bug Fixes

## References
* [multiprocessing Basics](https://pymotw.com/2/multiprocessing/basics.html)
* [CORS Home](https://geodesy.noaa.gov/CORS/)
* [IGS (RINEX) File formats](https://kb.igs.org/hc/en-us/articles/201096516-IGS-Formats)
* [User Friendly CORS](https://www.ngs.noaa.gov/UFCORS/)
* [CORS Map](https://geodesy.noaa.gov/CORS_Map/)
* [Sortable list of CORS Stations](https://geodesy.noaa.gov/CORS/sort_sites.shtml)
* [ANTEX Antenna file ngs14.atx](https://www.ngs.noaa.gov/ANTCAL/LoadFile?file=ngs14.atx)
* [Juypter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html)
* [Python subprocess](https://docs.python.org/3/library/subprocess.html)

## Todo list
* Add code to gather and merge "same day" data. [(see teqc merge post here)](https://postal.unavco.org/pipermail/teqc/2014/001827.html)
* Switch to [Juypter Widgets](https://ipywidgets.readthedocs.io/en/latest/index.html)
* Add [multiprocessing](https://pymotw.com/2/multiprocessing/basics.html) to gather all the stations in parallel.

## Change Log & Bug Fixes
* 2020-0622
  * Added input validity checking to time input strings.
  * Initial commit.